In [5]:
%pip install xgboost bentoml scikit-learn numpy fastai joblib typing

Defaulting to user installation because normal site-packages is not writeable
  Using cached xgboost-2.1.3-py3-none-manylinux_2_28_x86_64.whl (153.9 MB)
  Using cached bentoml-1.3.18-py3-none-any.whl (1.1 MB)
  Using cached scikit_learn-1.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
  Using cached numpy-2.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Using cached fastai-2.7.18-py3-none-any.whl (234 kB)
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Using cached typing-3.7.4.3.tar.gz (78 kB)
  Preparing metadata (setup.py) ... done
  Using cached nvidia_nccl_cu12-2.23.4-py3-none-manylinux2014_x86_64.whl (199.0 MB)
  Using cached scipy-1.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (40.6 MB)
  Using cached cloudpickle-3.1.0-py3-none-any.whl (22 kB)
  Using cached opentelemetry_instrumentation_asgi-0.50b0-py3-none-any.whl (16 kB)
  Using cached fs-2.4.16-py2.py3-none-any.whl (135 kB)
  Using cached questi

In [10]:
%pip install --upgrade scikit-learn xgboost bentoml

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
%pip install distro-info==1.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install streamlit

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 1.0 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 KB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 354.9 kB/s eta 0:00:0000:0100:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 KB 45.0 kB/s eta 0:00:00a 0:00:07
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 KB 143.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 KB 175.6 kB/s eta 0:00:0000:0100:04
  Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 KB 258.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.7/282.7 KB 181.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB

In [23]:
%pip install python-apt


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install scikit-learn==1.3.1

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 57.4 kB/s eta 0:00:0000:0100:06
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip freeze > requirements.txt

In [11]:
!python load_save.py

/home/rubanza/.local/lib/python3.10/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
/home/rubanza/.local/lib/python3.10/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as

In [18]:
%pip uninstall -y scikit-learn
%pip install scikit-learn==1.5.2

Found existing installation: scikit-learn 1.6.0
Uninstalling scikit-learn-1.6.0:
  Successfully uninstalled scikit-learn-1.6.0
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 664.4 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [19]:
!python test.py

/home/rubanza/.local/lib/python3.10/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
/home/rubanza/.local/lib/python3.10/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as

In [3]:
import numpy as np
from numpy import random
#import fastbook
#fastbook.setup_book()
#from fastbook import *
from fastai.tabular.all import *
import pandas as pd
#import matplotlib.pyplot as plt
#from fastai.imports import *
#np.set_printoptions(linewidth=130)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pathlib import Path
import os
import xgboost as xgb
#from xgboost import plot_importance
from xgboost import XGBClassifier
import warnings
import gc
import pickle
from joblib import dump, load
import typing as t
import bentoml


In [4]:
path = Path('data/')

train_df = pd.read_csv(path/'train.csv',index_col='id')

In [5]:
train_df.head()

,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
id,,,,,,,,,,,,,,,,,,,
0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [6]:
train_df['Profession'].unique(),train_df['Profession'].nunique()

(array(['Chef', 'Teacher', nan, 'Business Analyst', 'Finanancial Analyst',
        'Chemist', 'Electrician', 'Software Engineer', 'Data Scientist',
        'Plumber', 'Marketing Manager', 'Accountant', 'Entrepreneur',
        'HR Manager', 'UX/UI Designer', 'Content Writer',
        'Educational Consultant', 'Civil Engineer', 'Manager',
        'Pharmacist', 'Financial Analyst', 'Architect',
        'Mechanical Engineer', 'Customer Support', 'Consultant', 'Judge',
        'Researcher', 'Pilot', 'Graphic Designer', 'Travel Consultant',
        'Digital Marketer', 'Lawyer', 'Research Analyst',
        'Sales Executive', 'Doctor', 'Unemployed', 'Investment Banker',
        'Family Consultant', 'B.Com', 'BE', 'Student', 'Yogesh', 'Dev',
        'MBA', 'LLM', 'BCA', 'Academic', 'Profession', 'FamilyVirar',
        'City Manager', 'BBA', 'Medical Doctor', 'Working Professional',
        'MBBS', 'Patna', 'Unveil', 'B.Ed', 'Nagpur', 'Moderate', 'M.Ed',
        'Analyst', 'Pranav', 'Visakhapatn

In [13]:
train_df['Sleep Duration'].nunique(),train_df['Sleep Duration'].unique()

(36,
 array(['More than 8 hours', 'Less than 5 hours', '5-6 hours', '7-8 hours',
        'Sleep_Duration', '1-2 hours', '6-8 hours', '4-6 hours',
        '6-7 hours', '10-11 hours', '8-9 hours', '40-45 hours',
        '9-11 hours', '2-3 hours', '3-4 hours', 'Moderate', '55-66 hours',
        '4-5 hours', '9-6 hours', '1-3 hours', 'Indore', '45', '1-6 hours',
        '35-36 hours', '8 hours', 'No', '10-6 hours', 'than 5 hours',
        '49 hours', 'Unhealthy', 'Work_Study_Hours', '3-6 hours',
        '45-48 hours', '9-5', 'Pune', '9-5 hours'], dtype=object))

In [14]:
train_df['Degree'].nunique(),train_df['Degree'].unique()

(115,
 array(['BHM', 'LLB', 'B.Pharm', 'BBA', 'MCA', 'MD', 'BSc', 'ME', 'B.Arch',
        'BCA', 'BE', 'MA', 'B.Ed', 'B.Com', 'MBA', 'M.Com', 'MHM', 'BA',
        'Class 12', 'M.Tech', 'PhD', 'M.Ed', 'MSc', 'B.Tech', 'LLM',
        'MBBS', 'M.Pharm', 'UX/UI Designer', 'MPA', 'BH', 'Nalini', 'BEd',
        'B.Sc', 'Veda', 'Bhopal', 'S.Tech', 'Degree', '20', 'Class 11',
        'H_Pharm', 'M', 'P.Com', 'BPharm', 'Business Analyst', 'M.Arch',
        'LL.Com', 'Data Scientist', 'MPharm', 'L.Ed', 'P.Pharm', 'Kalyan',
        'Unite', 'BArch', 'HR Manager', 'Badhya', 'S.Pharm', 'LLBA',
        'Vrinda', 'M. Business Analyst', 'Bhavesh', '0', 'LLCom', '29',
        'MTech', 'Vivaan', 'BPA', 'Plumber', '5.61', 'Brit', 'B.03',
        'Ritik', '5.56', 'MEd', 'B', 'B BA', '7.06', 'B.B.Arch', 'ACA',
        'Brithika', 'CGPA', '24', 'M_Tech', 'Pihu', 'BB', 'Jhanvi',
        'LLTech', 'Aarav', 'Entrepreneur', '8.56', 'LHM', 'Lata', 'S.Arch',
        'Marsh', 'HCA', '5.88', 'B.Student', 'LL B.Ed',

In [15]:
train_df['Dietary Habits'].nunique(),train_df['Dietary Habits'].unique()


(23,
 array(['Healthy', 'Unhealthy', 'Moderate', 'Yes', 'Pratham', 'BSc',
        'Gender', '3', 'More Healthy', 'Less than Healthy', 'Mihir', '1.0',
        'Hormonal', 'Electrician', nan, 'No Healthy', 'Less Healthy',
        'M.Tech', 'Vegas', 'No', 'Male', 'Indoor', 'Class 12', '2'],
       dtype=object))